In [13]:
# Needed for functions
import numpy as np
import time
from copy import deepcopy

# Import Qiskit classes
import qiskit
import qiskit.quantum_info as qi
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister, Aer
from qiskit.providers.aer import noise
from qiskit.compiler import assemble

# Tomography functions
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
from qiskit.ignis.verification.tomography import process_tomography_circuits, ProcessTomographyFitter
from qiskit.ignis.verification.tomography import gateset_tomography_circuits, GatesetTomographyFitter
import qiskit.ignis.mitigation.measurement as mc

# Auxiliary methods
from qiskit.quantum_info import Choi, Kraus
from qiskit.extensions import HGate, XGate

In [37]:
def rand_angles():
    return tuple(2 * np.pi * np.random.random(3) - np.pi)

nq = 1
shots = 1000

q = QuantumRegister(nq)
circ = QuantumCircuit(q)
for j in range(nq):
    circ.u(*rand_angles(), q[j])


target_state = qi.Statevector.from_instruction(circ)

qst_circs = state_tomography_circuits(circ, q)
print(qst_circs[2].draw())
print(circ.draw())
job = qiskit.execute(qst_circs, Aer.get_backend('qasm_simulator'),
                        shots=shots)
tomo_data = StateTomographyFitter(job.result(), qst_circs)
rho_fit = tomo_data.fit(method='lstsq')

print('F = {:.5f}'.format(qi.state_fidelity(rho_fit, target_state)))

        ┌──────────────────────────┐ ░ ┌─┐
q924_0: ┤ U(0.59859,1.9439,1.3072) ├─░─┤M├
        └──────────────────────────┘ ░ └╥┘
  c9: 1/════════════════════════════════╩═
                                        0 
        ┌──────────────────────────┐
q924_0: ┤ U(0.59859,1.9439,1.3072) ├
        └──────────────────────────┘
F = 0.99946
